In [2]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [4]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end ")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [5]:
openai = OpenAI()

In [6]:
message = "Hello, GPT!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [10]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        #Create this Website object from the given url using the BeautifulSoup library
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [11]:
ws = Website("https://bbc.com")
print(ws.title)
print(ws.text)

BBC Home - Breaking News, World News, US News, Sports, Business, Innovation, Climate, Culture, Travel, Video & Audio
Skip to content
British Broadcasting Corporation
Home
News
Sport
Business
Innovation
Culture
Arts
Travel
Earth
Audio
Video
Live
Home
News
Israel-Gaza War
War in Ukraine
US & Canada
UK
UK Politics
England
N. Ireland
N. Ireland Politics
Scotland
Scotland Politics
Wales
Wales Politics
Africa
Asia
China
India
Australia
Europe
Latin America
Middle East
In Pictures
BBC InDepth
BBC Verify
Sport
Business
Executive Lounge
Technology of Business
Future of Business
Innovation
Technology
Science & Health
Artificial Intelligence
AI v the Mind
Culture
Film & TV
Music
Art & Design
Style
Books
Entertainment News
Arts
Arts in Motion
Travel
Destinations
Africa
Antarctica
Asia
Australia and Pacific
Caribbean & Bermuda
Central America
Europe
Middle East
North America
South America
World’s Table
Culture & Experiences
Adventures
The SpeciaList
Earth
Natural Wonders
Weather & Science
Climate S

In [12]:
# Define our system prompt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [13]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [14]:
print(user_prompt_for(ws))

You are looking at a website titled BBC Home - Breaking News, World News, US News, Sports, Business, Innovation, Climate, Culture, Travel, Video & Audio
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Skip to content
British Broadcasting Corporation
Home
News
Sport
Business
Innovation
Culture
Arts
Travel
Earth
Audio
Video
Live
Home
News
Israel-Gaza War
War in Ukraine
US & Canada
UK
UK Politics
England
N. Ireland
N. Ireland Politics
Scotland
Scotland Politics
Wales
Wales Politics
Africa
Asia
China
India
Australia
Europe
Latin America
Middle East
In Pictures
BBC InDepth
BBC Verify
Sport
Business
Executive Lounge
Technology of Business
Future of Business
Innovation
Technology
Science & Health
Artificial Intelligence
AI v the Mind
Culture
Film & TV
Music
Art & Design
Style
Books
Entertainment News
Arts
Arts in Motion
Travel
Destinations
Africa
Antarctica
Asia
Australia a

In [15]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [16]:
#calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, let me consult my vast database of advanced mathematics... just kidding! It's 4. Shocking, I know!


In [17]:

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [18]:
messages_for(ws)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled BBC Home - Breaking News, World News, US News, Sports, Business, Innovation, Climate, Culture, Travel, Video & Audio\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nSkip to content\nBritish Broadcasting Corporation\nHome\nNews\nSport\nBusiness\nInnovation\nCulture\nArts\nTravel\nEarth\nAudio\nVideo\nLive\nHome\nNews\nIsrael-Gaza War\nWar in Ukraine\nUS & Canada\nUK\nUK Politics\nEngland\nN. Ireland\nN. Ireland Politics\nScotland\nScotland Politics\nWales\nWales Politics\nAfrica\nAsia\nChina\nIndia\nAustralia\nEurope\nLatin America\nMiddle East\nIn Pictures\nBBC InDepth\nBBC Verify\nSport\nBusiness\nExecu

In [19]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [20]:
summarize("https://bbc.com")

"# BBC Home Overview\n\nThe **BBC Home** website serves as a comprehensive platform for global news coverage, sports, business insights, innovation, cultural events, and environmental issues. \n\n## News Highlights\n- **Israel-Gaza War** and **War in Ukraine** are ongoing critical topics.\n- Recent coverage includes:\n  - **Trump's Speech Fact-Check**: BBC Verify examined claims made by the US president regarding various political issues.\n  - **Canada's Conservative Party**: Young voters showed increased support for them in the recent elections, creating challenges for the current minority government.\n  - **Natural Resources Deal**: The US signed a significant agreement with Ukraine after protracted negotiations.\n  - **Endangered Axolotl**: There is optimism surrounding the release of this rare amphibian, previously on the brink of extinction.\n  - **NFL Fines**: The Atlanta Falcons were penalized due to prank calls made during the NFL Draft.\n  \n## Environment & Science\n- The UK 

In [21]:
# display this nicely using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [22]:
display_summary("https://bbc.com")

# BBC Home Summary

The BBC Home website provides a comprehensive overview of global news, sports, business updates, and cultural content. Key sections include:

## Latest News Highlights
- **Israel-Gaza War:** Ongoing coverage on the conflict.
- **US & Canada:**
  - **Trump's 100 Days Speech:** Claims fact-checked by BBC Verify, highlighting discrepancies in Trump's recent statements.
  - **Young Voters' Shift:** Analysis of why young voters are gravitating towards Canada’s Conservatives despite their electoral loss.
  - **Historical Resource Deal:** A landmark natural resources agreement signed between the US and Ukraine after lengthy negotiations.
  - **Military Tensions:** Pakistan warns of possible military strikes from India following rising tensions after an attack on tourists in Kashmir.

## Environmental and Health Updates
- **Endangered Axolotl:** Efforts to revive this rare amphibian are discussed amidst concerns of extinction in Mexico.
- **Climate and Nature:** Coverage includes the aftermath of major environmental events and critiques of UK climate policies.

## Sports
- Highlights from various sports, including:
  - NFL fines for a prank call incident involving the Atlanta Falcons.
  - Lamine Yamal, a young football talent receiving acclaim.
  
## Business
- Economic updates indicating a contraction in the US economy.
- Discussions surrounding tariffs and their implications on trade between the US and China.

## Culture and Travel
- Articles exploring diverse cultural topics, including art, film critiques, and unique travel destinations.

Overall, the BBC Home serves as a reliable source for breaking news and in-depth analysis across multiple domains including politics, economy, environment, and cultural experiences.